# Preliminary Analysis

Before pre processing is performed, lets examine and derive some insight from the datasets provided. Although, some carry-over preprocess will be necessary. 

In [14]:
import pandas as pd
# Load the datasets
draft_data = pd.read_csv('../data/raw/draft_history.csv',  index_col=[0])
position_history = pd.read_csv('../data/raw/game_positions.csv',  index_col=[0])
player_stats = pd.read_csv('../data/raw/player_statistics.csv',  index_col=[0])
afltable_hist = pd.read_csv('../data/raw/afltables_draft.csv',  index_col=[0])

/tmp/ipykernel_262/2855217957.py:4: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  position_history = pd.read_csv('../data/raw/game_positions.csv',  index_col=[0])


In [2]:
# first look at draft_data
draft_data
# the first thing to notice, is that year data and draft position is treated as double/float, 
# as well as 'recruitedFrom' is a multi value attribute, that needs to be processed
# draft_data.dtypes (also can use this)

,firstName,surname,id,team,season,jumperNumber,position,providerId,dateOfBirth,draftYear,heightInCm,weightInKg,recruitedFrom,debutYear,draftType,draftPosition,data_accessed
1,Scott,Thompson,2,Adelaide Crows,2013,5,MIDFIELDER,CD_I210016,1983-03-14,2000.0,185,86,North Haven (SA)/Port Adelaide (SANFL)/Melbourne,2001.0,nationalDraft,16.0,2022-08-26
2,Graham,Johncock,1132,Adelaide Crows,2013,18,MEDIUM_FORWARD,CD_I210056,1982-10-21,2000.0,178,82,Port Adelaide (SANFL),2002.0,nationalDraft,67.0,2022-08-26
3,Brent,Reilly,31,Adelaide Crows,2013,3,MEDIUM_DEFENDER,CD_I220086,1983-11-12,2001.0,183,84,Gisborne (Vic)/Calder U18,2002.0,nationalDraft,12.0,2022-08-26
4,Ben,Rutten,43,Adelaide Crows,2013,25,KEY_DEFENDER,CD_I220093,1983-05-28,2002.0,190,98,West Adelaide (SA),2003.0,rookieElevation,NaN,2022-08-26
5,Jason,Porplyzia,3,Adelaide Crows,2013,40,MEDIUM_FORWARD,CD_I230174,1984-11-27,2005.0,179,80,West Adelaide (SA),2006.0,preseason,9.0,2022-08-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8183,Josh,Rotham,1380,West Coast Eagles,2022,35,MEDIUM_DEFENDER,CD_I997823,1998-02-25,2016.0,193,89,Whitford (WA)/West Perth (WAFL),2019.0,nationalDraft,37.0,2022-08-26
8184,Alex,Witherden,1307,West Coast Eagles,2022,23,MEDIUM_DEFENDER,CD_I998128,1998-09-10,2016.0,188,86,St Joseph&apos;s (Vic)/Geelong U18/Brisbane,2017.0,nationalDraft,23.0,2022-08-26
8185,Luke,Strnadica,1391,West Coast Eagles,2022,45,RUCK,CD_I998192,1998-01-01,2021.0,202,102,Winnacott (WA)/ E Frem (WAFL)/Fremantle list/E...,2022.0,preseason,NaN,2022-08-26
8186,Jamaine,Jones,1300,West Coast Eagles,2022,31,MEDIUM_DEFENDER,CD_I998791,1998-09-29,2019.0,179,81,Heywood (Vic)/Portland (Vic)/North Ballarat U1...,2018.0,preseason,NaN,2022-08-26


In [3]:
set(draft_data['draftType'].values)

{'fatherSon',
 nan,
 'nationalDraft',
 'other',
 'preseason',
 'rookieElevation',
 'zone'}

In [4]:
# lets look at null values
draft_data.isnull().any()
# we can see that draftYear, recruitedFrom, debutYear, draftType and draftPosition include null values

firstName        False
surname          False
id               False
team             False
season           False
jumperNumber     False
position         False
providerId       False
dateOfBirth      False
draftYear         True
heightInCm       False
weightInKg       False
recruitedFrom     True
debutYear         True
draftType         True
draftPosition     True
data_accessed    False
dtype: bool

The following statements can be said about the following null value attributes: 
- draftYear: I believe these have occured due to the introduction of mid-season drafts, ssp, etc. drafts, which have not been registered to a year. In saying this, i believe these attributes can be derived from from the season attribute for year after 2013 (due to new players only being added in supplimantary years). Therefore players with this null value recorded in 2013 will be removed. 
- recruitedFrom: simply can be replaced with 'UNKNOWN'
- debutYear: There is a potential to it referring to players who never played a game, but it also has a high correlation with draftYear being missing

In [5]:
draft_data[(draft_data['debutYear'].isnull()) & (draft_data['draftYear'].isnull())]

,firstName,surname,id,team,season,jumperNumber,position,providerId,dateOfBirth,draftYear,heightInCm,weightInKg,recruitedFrom,debutYear,draftType,draftPosition,data_accessed
22,Shaun,McKernan,17,Adelaide Crows,2013,35,KEY_FORWARD,CD_I280038,1990-09-01,NaN,198,98,NaN,NaN,NaN,NaN,2022-08-26
92,Nick,Hayes,246,Brisbane Lions,2013,43,KEY_FORWARD,CD_I296937,1994-08-11,NaN,189,82,NaN,NaN,NaN,NaN,2022-08-26
134,Michael,Hartley,907,Collingwood,2013,47,KEY_DEFENDER,CD_I294077,1993-06-07,NaN,198,102,NaN,NaN,NaN,NaN,2022-08-26
138,Jackson,Ramsay,265,Collingwood,2013,31,MEDIUM_DEFENDER,CD_I295827,1994-11-20,NaN,184,82,NaN,NaN,NaN,NaN,2022-08-26
167,Brandon,Matera,336,Gold Coast Suns,2013,32,MEDIUM_FORWARD,CD_I290817,1992-03-11,NaN,173,74,NaN,NaN,NaN,NaN,2022-08-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7375,Daniel,Venables,1470,West Coast Eagles,2021,18,MEDIUM_FORWARD,CD_I998414,1998-11-19,NaN,187,84,NaN,NaN,NaN,NaN,2022-08-26
7536,Oskar,Faulkhead,5038,Gold Coast Suns,2022,37,MIDFIELDER,CD_I1017089,2003-02-11,NaN,182,76,NaN,NaN,NaN,NaN,2022-08-26
7765,Max,Ramsden,6683,Hawthorn,2022,38,RUCK,CD_I1027965,2003-04-19,NaN,203,86,NaN,NaN,NaN,NaN,2022-08-26
7872,Massimo,D'Ambrosio,4349,Essendon,2022,42,MEDIUM_DEFENDER,CD_I1005144,2003-06-05,NaN,178,80,NaN,NaN,NaN,NaN,2022-08-26


In [6]:
# firstly, lets change the datastypes
draft_data.astype({'draftYear': int, 'debutYear': int, 'draftPosition': int})


IntCastingNaNError: Cannot convert non-finite values (NA or inf) to integer